Name: Ali Oku <br>
Z-ID: Z1893417 <br>
Assignment 6<br>

Schedule Class

In [14]:
class Schedule:
    
    def __init__(self, s_list=None):
        if s_list == None:
            self.list_of_courses = []
            
        else:
            self.list_of_courses = s_list
            
            
    @property
    def credits(self):
        total_credits = sum([courses.credits for courses in self.list_of_courses])
        return total_credits
    
    def add_course(self, new_course):
        for course in self.list_of_courses:
            if course & new_course == True:
                raise Exception((f"Course cannot be added because {new_course} conflicts with {course} in the current schedule"))
                

Academic Classes

In [15]:
class Academic:
    def __init__(self, campus_id, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        self.campus_id = campus_id
        self.max_credits = self.MAX_CREDITS
        self.schedule = Schedule()
        
    def add_course(self, course):
        if self.schedule.credits + course.credits > self.max_credits:
            raise Exception((f"Exceeded maximum credits of {self.max_credits} credits"))
                               
        if self.schedule.credits + course.credits <= self.max_credits:
            self.schedule.add_course(course)
            
        self.schedule.list_of_courses.append(course)
                
    
    def remove_course(self,course):
        self.schedule.list_of_courses.remove(course)
            
        
class Student(Academic):
    MAX_CREDITS = 16
    
    def __init__(self, campus_id, first_name, last_name, level):
        super().__init__(campus_id,first_name, last_name)
        self.level = level 
        
    def add_course(self,course):
        super().add_course(course)
        course.enroll(self)
        
    def remove_course(self,course):
        super().remove_course(course)
        course.drop(self)
    
    def print_schedule(self):
        print(f'{"Department": ^10}' + 
      f'{"Course Number": ^17}' + 
      f'{"Course Name": ^35}' +
      f'{"Section": ^10}' +
      f'{"Credits": ^10}' + 
      f'{"Instructor": ^25}' +
      f'{"Meeting Times": ^25}')
              
        for course in self.schedule.list_of_courses:
            print(f'{course.department:^10}' + 
          f'{course.course_number: ^17}' + 
          f'{course.course_name: ^35}' + 
          f'{course.section: ^10}' + 
          f'{course.credits: ^10}' + 
          f'{course.instructor_name : ^25}', course.times)
        
class GraduateStudent(Student):
    MAX_CREDITS = 12
    
    def __init__(self, campus_id, first_name, last_name):
        super().__init__(campus_id, first_name, last_name, "Graduate")
        
class Instructor(Academic):
    MAX_CREDITS = 9
    
    def __init__(self, campus_id, first_name, last_name,rank):
        super().__init__(campus_id,first_name, last_name)
        self.rank = rank   
        
    def add_course(self,course):
        super().add_course(course)
        course.instructor = self
        course.instructor_name = str(self.first_name) + " " + str(self.last_name)
        
    def remove_course(self,course):
        super().remove_course(course)
        course.instructor = "N/A"
        
    def print_schedule(self):
        print(f'{"Department": ^10}' + 
      f'{"Course Number": ^17}' + 
      f'{"Course Name": ^35}' +
      f'{"Section": ^10}' +
      f'{"Credits": ^10}' + 
      f'{"Meeting Times": ^25}')
     
        for course in self.schedule.list_of_courses:
            print(f'{course.department:^10}' + 
          f'{course.course_number: ^17}' + 
          f'{course.course_name: ^35}' + 
          f'{course.section: ^10}' + 
          f'{course.credits: ^10}', course.times)
    

In [16]:
class Course:
    
    def __init__(self, department, course_number, course_name, section, credits, times, instructor = "N/A"):
        self.instructor = instructor
        self.department = department 
        self.course_number = course_number 
        self.course_name = course_name 
        self.section = section
        self.credits = credits 
        self.times = times
        self.students_enrolled = []      #tracks the list of students enrolled in the course
        
        if instructor == "N/A":
            self.instructor_name = self.instructor 
        else:
            self.instructor_name = str(self.instructor.first_name) + " " + str(self.instructor.last_name)
    
    def __str__(self):
        return f'{self.department} {self.course_number} {self.section} {self.times}'
    
    @staticmethod 
    def check_times_conflict(time_1, time_2):
        conflict = False
        for item_1 in time_1:
            for item_2 in time_2:
                if item_1[0] == item_2[0]:
                    if item_1[1] <= item_2[1] < item_1[2]:
                        conflict = True
                        return conflict
                    elif item_2[1] < item_1[1] and item_2[2] > item_1[1]:
                        conflict = True
                        return conflict
            
        return conflict 
                
                
    def __and__ (self, course):
        if self.check_times_conflict(self.times, course.times) == True:
            return True
        
        else:
            return False
        
    def enroll(self, student):      #enrolls student into the course
        self.students_enrolled.append(student)
        
    
    def drop(self, student):
        self.students_enrolled.remove(student)
        
    def change_time(self, new_time):
        for student in self.students_enrolled:
            for course in student.schedule.list_of_courses:
                if course.check_times_conflict(course.times, new_time) == True:
                    raise Exception((f"Course time cannot changed be because {new_time} conflicts with a student's current schedule"))
                    
        if self.instructor == "N'A":
            pass
        else:
            for course in self.instructor.schedule.list_of_courses:
                if course.check_times_conflict(course.times, new_time) == True:
                    raise Exception((f"Course time cannot be changed because {new_time} conflicts with a instructors's current schedule"))
                    
        self.times = new_time
        
        
    def print_course_enrollment(self):
        print(f'{"Instructor:": <20}' + f'{self.instructor_name: <20}' + "\n")
        print(f'{"First Name": <15}' +
              f'{"Last Name": <15}' +
              f'{"Student ID": <15}' + "\n")
              
        for student in self.students_enrolled:
            print (f'{student.first_name: <15}'+
                   f'{student.last_name: <15}'+
                   f'{student.campus_id: <15}')
                   
              
        


In [17]:
class Registrar:
    
    def __init__(self):
        self.list_of_academics = {}
        self.list_of_courses = {}
        
    def add_persons(self, academics):
        for academic in academics:
            self.list_of_academics[academic.campus_id] = academic
    
    def add_courses(self, courses):
        for course in courses:
            self.list_of_courses[course] = [course.department, course.course_number, course.section]
        
    def add_person_to_course(self, campus_id, department, course_number, section):
        for campus_ids in self.list_of_academics.keys():
            if campus_ids == campus_id:
                academic = self.list_of_academics[campus_ids]
        
        for courses, course_info in self.list_of_courses.items():
            if course_info == [department, course_number, section]:
                course = courses
        
        academic.add_course(course)
                            
    
    def remove_person_from_course(self, campus_id, department, course_number, section):
        for campus_ids in self.list_of_academics.keys():
            if campus_ids == campus_id:
                academic = self.list_of_academics[campus_ids]
        
        for courses, course_info in self.list_of_courses.items():
            if course_info == [department, course_number, section]:
                course = courses
        
        academic.remove_course(course)
        
        
    
    def change_course_time(self, department, course_number, section, new_time):
        for courses, course_info in self.list_of_courses.items():
            if course_info == [department, course_number, section]:
                course = courses
        
        course.change_time(new_time)
        